In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from data_loader import data_label_split
from data_loader import generate_data_set
from data_loader import dmso_taxol_ProfileBag
from deepset_exp import mini_noise_signal_cv
from deepset_exp import train
from deepset_exp import test

from model import SmallDeepSet
from model import FullDeepSet, profile_AttSet

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

import torch
import torch.utils.data as D 
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

In [2]:
drop_NA_data = pd.read_csv("moa_data_drop_NA.csv", index_col=0)
sf_drop_NA_data = drop_NA_data[["compound", "concentration",
                                "moa", "row ID", "Iteration (#2)", "COND",
                               "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [3]:
mini_data = pd.read_csv("mini_moa_data_drop_NA.csv",index_col=0)
# sf_drop_NA_data = mini_data[["compound", "concentration",
#                                 "moa", "row ID", "Iteration (#2)", "COND",
#                                "AreaShape_Area_Nuclei", "AreaShape_Compactness_Nuclei"]]

In [4]:

# bagData = dmso_taxol_ProfileBag(sf_drop_NA_data, 200, 50, 5, 0.5, "taxol", "DMSO", 0.5)
# dataloader = D.DataLoader(bagData, batch_size=1, shuffle=True)

In [5]:
# data = sf_drop_NA_data
# num_bag = 10
# bag_size_mean = 10
# bag_size_std = 0
# treatment = "taxol" 
# control = "DMSO"
# model = sf_deepset
# optimizer = optimizer
# splits = 4
# epochs = 1

# Deep Set Model

In [6]:
full_deepset = FullDeepSet().cuda()

In [130]:
sf_deepset = SmallDeepSet().cuda()

In [7]:
X,y=data_label_split(drop_NA_data)
data = pd.concat([pd.DataFrame(normalize(X), columns = X.columns), y.reset_index(drop=True)], axis=1, sort = False )


In [25]:
start = 90
end = 91
# data = drop_NA_data
num_bag = 100
bag_size_mean = 200
bag_size_std = 10
treatment = "taxol"
control = "DMSO"
model = FullDeepSet("mean").cuda()
# optimizer = optimizer
splits = 3
epochs = 20

In [19]:
model.reset

FullDeepSet(
  (enc): Sequential(
    (0): Linear(in_features=481, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
  )
  (dec): Sequential(
    (0): Linear(in_features=64, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [20]:

for layer in model.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.999), weight_decay=10e-5)

In [29]:
results = mini_noise_signal_cv(start, end, data, num_bag,
                               bag_size_mean, bag_size_std, treatment, control, model, optimizer, splits, epochs)

training at different percent:   0%|          | 0/1 [00:00<?, ?it/s]
3 fold cross validation: 0it [00:00, ?it/s]

For 0 epoch: 0it [00:00, ?it/s]

For 0 epoch: 6it [00:00, 53.99it/s]

For 0 epoch: 13it [00:00, 53.49it/s]

For 0 epoch: 20it [00:00, 56.59it/s]

For 0 epoch: 27it [00:00, 59.06it/s]

For 0 epoch: 35it [00:00, 62.72it/s]

For 0 epoch: 43it [00:00, 65.07it/s]

For 0 epoch: 50it [00:00, 66.30it/s]

For 0 epoch: 57it [00:00, 67.20it/s]

For 0 epoch: 66it [00:01, 65.64it/s]


For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 8it [00:00, 68.69it/s]

Epoch: 0, Loss: 0.6942, Train error: 0.5000




For 1 epoch: 15it [00:00, 66.29it/s]

For 1 epoch: 21it [00:00, 61.91it/s]

For 1 epoch: 29it [00:00, 64.72it/s]

For 1 epoch: 36it [00:00, 65.86it/s]

For 1 epoch: 44it [00:00, 68.72it/s]

For 1 epoch: 51it [00:00, 68.98it/s]

For 1 epoch: 66it [00:00, 68.74it/s]


For 2 epoch: 0it [00:00, ?it/s]

For 2 epoch: 6it [00:00, 52.06it/s]

Epoch: 1, Loss: 0.6942, Train error: 0.5000




For 2 epoch: 13it [00:00, 54.88it/s]

For 2 epoch: 19it [00:00, 56.30it/s]

For 2 epoch: 26it [00:00, 57.75it/s]

For 2 epoch: 33it [00:00, 58.54it/s]

For 2 epoch: 41it [00:00, 63.42it/s]

For 2 epoch: 48it [00:00, 63.72it/s]

For 2 epoch: 55it [00:00, 62.61it/s]

For 2 epoch: 66it [00:01, 63.10it/s]


For 3 epoch: 0it [00:00, ?it/s]

For 3 epoch: 7it [00:00, 62.35it/s]

...



For 4 epoch: 13it [00:00, 62.37it/s]

For 4 epoch: 20it [00:00, 62.66it/s]

For 4 epoch: 26it [00:00, 61.55it/s]

For 4 epoch: 33it [00:00, 62.20it/s]

For 4 epoch: 39it [00:00, 61.04it/s]

For 4 epoch: 46it [00:00, 62.25it/s]

For 4 epoch: 53it [00:00, 62.95it/s]

For 4 epoch: 59it [00:00, 59.22it/s]

For 4 epoch: 66it [00:01, 57.96it/s]


For 5 epoch: 0it [00:00, ?it/s]

For 5 epoch: 8it [00:00, 69.93it/s]

Epoch: 4, Loss: 0.6941, Train error: 0.5000




For 5 epoch: 15it [00:00, 69.01it/s]

For 5 epoch: 22it [00:00, 67.29it/s]

For 5 epoch: 30it [00:00, 68.69it/s]

For 5 epoch: 36it [00:00, 65.22it/s]

For 5 epoch: 43it [00:00, 66.25it/s]

For 5 epoch: 50it [00:00, 65.27it/s]

For 5 epoch: 56it [00:00, 52.88it/s]

For 5 epoch: 66it [00:01, 59.22it/s]


Epoch: 5, Loss: 0.6941, Train error: 0.5000



3 fold cross validation: 3it [00:34, 11.49s/it]
training at different percent:   0%|          | 0/1 [00:34<?, ?it/s]


In [11]:

results

{0.9: [1.0,
  0.0,
  1.0,
  0.0,
  0.75,
  0.4330127018922193,
  0.00343812870493161,
  0.005264424610017966,
  0.8045719616735975,
  0.2824623806185282]}

In [12]:
dic = {}
# Set different percentage of treatment v.s. control 
for j in tqdm(range(start, end, 5), desc = "training at different percent"):
    X, y = data_label_split(data)
    y = y["compound"]

    acc_control_list = []
    acc_treat_list = []
    pred_score_control_list = []
    pred_score_treat_list = []
    # Stratified K fold 
    skf = StratifiedKFold(n_splits = splits)
    for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y)), desc="%d fold cross validation"%splits):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]  
        X_train = pd.concat([X_train, y_train], axis=1, sort=False)
        X_test = pd.concat([X_test, y_test], axis=1, sort=False)

        # Redefine dataloader and train model at each fold
        train_dataset = dmso_taxol_ProfileBag(X_train, int(num_bag*(splits-1)/splits), bag_size_mean, bag_size_std, j/100, treatment, control, 0.5)
        valida_dataset = dmso_taxol_ProfileBag(X_test, int(num_bag/splits), bag_size_mean, bag_size_std, j/100, treatment, control, 0.5)
        train_loader = D.DataLoader(train_dataset, batch_size=1, shuffle=True)
        valida_loader = D.DataLoader(valida_dataset, batch_size=1, shuffle=True)
        if i == 0:
            for epoch in range(int(epochs*2/3)):
                train(epoch, train_loader, model, optimizer)
        else:
            for epoch in range(int(epochs/3)):
                train(epoch, train_loader, model, optimizer)
        acc_control, acc_treat, pred_score_control, pred_score_treat = test(model, valida_loader)

        acc_control_list+=acc_control
        acc_treat_list+=acc_treat
        pred_score_control_list+=pred_score_control
        pred_score_treat_list+=pred_score_treat



For 0 epoch: 0it [00:00, ?it/s]

For 0 epoch: 5it [00:00, 47.79it/s]

For 0 epoch: 11it [00:00, 45.95it/s]

For 0 epoch: 15it [00:00, 43.22it/s]

For 0 epoch: 20it [00:00, 43.30it/s]

For 0 epoch: 24it [00:00, 40.99it/s]

For 0 epoch: 28it [00:00, 40.46it/s]

For 0 epoch: 33it [00:00, 41.43it/s]

For 0 epoch: 39it [00:00, 44.20it/s]

For 0 epoch: 44it [00:01, 44.68it/s]

For 0 epoch: 49it [00:01, 43.09it/s]

For 0 epoch: 55it [00:01, 45.33it/s]

For 0 epoch: 66it [00:01, 45.59it/s]


For 1 epoch: 0it [00:00, ?it/s]

For 1 epoch: 7it [00:00, 67.40it/s]

Epoch: 0, Loss: 0.0137, Train error: 0.0000




For 1 epoch: 14it [00:00, 65.00it/s]

For 1 epoch: 21it [00:00, 64.13it/s]

For 1 epoch: 28it [00:00, 63.66it/s]

For 1 epoch: 35it [00:00, 63.62it/s]

For 1 epoch: 42it [00:00, 64.11it/s]

For 1 epoch: 49it [00:00, 64.35it/s]

For 1 epoch: 55it [00:00, 62.06it/s]

For 1 epoch: 66it [00:01, 62.24it/s]


For 2 epoch: 0it [00:00, ?it/s]

For 2 epoch: 8it [00:00, 68.78it/s]

Epoch: 1, Loss: 0.0012, Train error: 0.0000




For 2 epoch: 15it [00:00, 66.89it/s]

For 2 epoch: 22it [00:00, 65.60it/s]

For 2 epoch: 29it [00:00, 65.60it/s]

For 2 epoch: 36it [00:00, 64.89it/s]

For 2 epoch: 42it [00:00, 63.23it/s]

For 2 epoch: 49it [00:00, 62.94it/s]

For 2 epoch: 55it [00:00, 56.53it/s]

For 2 epoch: 66it [00:01, 59.60it/s]


For 3 epoch: 0it [00:00, ?it/s]

For 3 epoch: 6it [00:00, 54.45it/s]

...

Epoch: 3, Loss: 0.0001, Train error: 0.0000




For 4 epoch: 15it [00:00, 71.39it/s]

For 4 epoch: 21it [00:00, 64.49it/s]

For 4 epoch: 26it [00:00, 58.37it/s]

For 4 epoch: 33it [00:00, 59.24it/s]

For 4 epoch: 39it [00:00, 57.59it/s]

For 4 epoch: 45it [00:00, 54.33it/s]

For 4 epoch: 52it [00:00, 56.62it/s]

For 4 epoch: 66it [00:01, 60.29it/s]


For 5 epoch: 0it [00:00, ?it/s]

For 5 epoch: 7it [00:00, 67.30it/s]

Epoch: 4, Loss: 0.0002, Train error: 0.0000




For 5 epoch: 14it [00:00, 67.93it/s]

For 5 epoch: 21it [00:00, 67.57it/s]

For 5 epoch: 28it [00:00, 68.25it/s]

For 5 epoch: 35it [00:00, 68.12it/s]

For 5 epoch: 43it [00:00, 68.63it/s]

For 5 epoch: 50it [00:00, 68.12it/s]

For 5 epoch: 57it [00:00, 63.97it/s]

For 5 epoch: 66it [00:01, 64.88it/s]


Epoch: 5, Loss: 0.0002, Train error: 0.0000




In [13]:

for i in [acc_control_list,
        acc_treat_list,
        pred_score_control_list,
        pred_score_treat_list]:
    print(np.mean(i))

1.0
0.7083333333333334
0.002746387370565322
0.76131448118637


In [14]:
np.mean(acc_control_list+acc_control_list)

1.0

In [15]:
np.mean(acc_control_list)

1.0

In [16]:
np.mean(acc_control_list)

1.0

In [92]:
np.mean(np.array(train_dataset[0][1][1])=="taxol")

0.8938053097345132

In [95]:
train_dataset[0][1][0]

tensor([1.])

In [101]:
np.mean(np.array(train_dataset[49][1][1])=="taxol")

0.0

In [100]:
train_dataset[49][1][0]

tensor([0.])

In [104]:
acc_control_list[0:25]

[1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0]

In [40]:
np.mean(acc_control_list[0:25]+acc_treat_list[0:25])

0.98

In [44]:
np.mean(acc_treat_list)

0.6666666666666666

In [27]:
len(acc_control_list)

51

In [29]:
np.mean(acc_control_list[0:18])

0.05555555555555555

In [30]:
np.mean(acc_treat_list[0:18])

0.8888888888888888

In [25]:
len(acc_control_list)/10

150.0

In [24]:
len(acc_treat_list)/10

150.0

In [42]:
np.mean(acc_control_list+acc_treat_list)

0.301

In [9]:
results = mini_noise_signal_cv(drop_NA_data, 1000, 100, 10, "taxol", "DMSO", full_deepset, 10, 50)

NameError: name 'drop_NA_data' is not defined

In [ ]:
sf_results = mini_noise_signal_cv(start = 90,
                                  end = 91,
                                  data = sf_drop_NA_data, 
                                  num_bag = 50,
                                  bag_size_mean = 200, 
                                  bag_size_std = 10, 
                                  treatment = "taxol", 
                                  control = "DMSO", 
                                  model = sf_deepset, 
                                  optimizer = optimizer,
                                  splits = 5,
                                  epochs = 2)

In [ ]:
sf_results

In [246]:
acc_control, acc_treat, pred_score_control, pred_score_treat = test(sf_deepset, dataloader)

In [29]:
(mean_control_accuracy, std_control_accuracy, 
            mean_treat_accuracy, std_treat_accuracy, 
            mean_pred_score_control, std_pred_score_control,
            mean_pred_score_treatment, std_pred_score_treatment) = sf_results

In [ ]:
dict = {'control_accuracy': nme, 'std_control_accuracy': deg, 'score': scr}  

In [24]:
sf_results = pd.DataFrame(list(sf_results)).transpose()

In [25]:
sf_results.columns = ["mean_control_accuracy", "std_control_accuracy", 
            "mean_treat_accuracy", "std_treat_accuracy", 
            "mean_pred_score_control", "std_pred_score_control",
            "mean_pred_score_treatment", "std_pred_score_treatment"]

In [26]:
sf_results

,mean_control_accuracy,std_control_accuracy,mean_treat_accuracy,std_treat_accuracy,mean_pred_score_control,std_pred_score_control,mean_pred_score_treatment,std_pred_score_treatment
0,0.6,0.489898,0.4,0.489898,0.392564,0.385677,0.393943,0.422387
1,0.1,0.300000,1.0,0.000000,0.722624,0.191698,0.814537,0.104709
2,0.4,0.489898,0.7,0.458258,0.543045,0.198318,0.673363,0.171835
3,0.9,0.300000,0.3,0.458258,0.213512,0.131968,0.393299,0.203889
4,0.9,0.300000,0.3,0.458258,0.187630,0.155302,0.295522,0.244676
5,0.7,0.458258,0.6,0.489898,0.324578,0.361401,0.679215,0.287370
6,0.8,0.400000,0.9,0.300000,0.343598,0.340132,0.728700,0.192789
7,1.0,0.000000,0.8,0.400000,0.134028,0.126734,0.733124,0.210020
8,0.6,0.489898,0.7,0.458258,0.386999,0.428574,0.691723,0.349524
9,0.7,0.458258,1.0,0.000000,0.397993,0.306416,0.989802,0.014643


In [38]:
import torch
import torch.nn.functional as F

In [11]:
x = torch.unsequeeze(torch.rand(10,2))

AttributeError: module 'torch' has no attribute 'unsequeeze'

In [54]:
x = torch.rand(4,10,481)

In [25]:
model = profile_AttSet(481)

In [27]:
model(x)

(tensor([[0.5026]], grad_fn=<SigmoidBackward>), tensor([[1.]]))

In [32]:
a,b = model.forward(x)

In [41]:
H = x.squeeze(0)
H = H.view(-1, model.input_feature)
H = model.feature_extractor_part2(H)  # NxL

A = model.attention(H)  # NxK
A = torch.transpose(A, 1, 0)  # KxN
A = F.softmax(A, dim=1)  # softmax over N
M = torch.mm(A, H)  # KxL

Y_prob = model.classifier(M)
Y_hat = torch.ge(Y_prob, model.thres).float()

In [63]:
H.shape

torch.Size([10, 481])

In [44]:
H.shape

torch.Size([10, 481])

In [15]:
SmallDeepSet()(x)

(tensor([[0.4996],
         [0.5000],
         [0.4998],
         [0.5001]], grad_fn=<SigmoidBackward>),
 tensor([[0],
         [0],
         [0],
         [1]], dtype=torch.uint8))

In [57]:
nn.Linear(481, 64)(x).shape

torch.Size([4, 10, 64])

In [58]:
nn.Linear(481, 64)(x).shape

torch.Size([4, 10, 64])

In [64]:
nn.Linear(481, 64)(x).bmm(nn.Linear(481, 64)(x).transpose(1,2)).bmm(nn.Linear(481, 64)(x)).shape

torch.Size([4, 10, 64])

In [61]:
nn.Linear(481, 64)(x).transpose(1,2).shape

torch.Size([4, 64, 10])

In [45]:

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Linear(dim_V, dim_V)

    def forward(self, Q, K):
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)

        dim_split = self.dim_V // self.num_heads
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)

        A = torch.softmax(Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V), 2)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + F.relu(self.fc_o(O))
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln)

    def forward(self, X):
        return self.mab(X, X)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln)

    def forward(self, X):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln)

    def forward(self, X):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X)

In [48]:

class SmallSetTransformer(nn.Module):
    def __init__(self,):
        super().__init__()
        self.enc = nn.Sequential(
            SAB(dim_in=481, dim_out=64, num_heads=4),
            SAB(dim_in=64, dim_out=64, num_heads=4),
        )
        self.dec = nn.Sequential(
            PMA(dim=64, num_heads=4, num_seeds=1),
            nn.Linear(in_features=64, out_features=1),
        )

    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        return x.squeeze(-1)



In [67]:
PMA(dim=64, num_heads=4, num_seeds=1)(SAB(dim_in=481, dim_out=64, num_heads=4)(x)).shape

torch.Size([4, 1, 64])

In [69]:
import random

In [70]:
random.shuffle(x)

In [55]:
SmallSetTransformer()(x)

tensor([[-2.3909e-02],
        [ 2.5366e-02],
        [ 2.7236e-02],
        [ 5.3927e-05]], grad_fn=<SqueezeBackward1>)

In [53]:
x.shape

torch.Size([1, 10, 481])